ref: https://github.com/NetManAIOps/OmniAnomaly

In [13]:
import os, sys, math

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [14]:
os.chdir('/home/maradonam/Implementation/Anomaly/TAD-main/')
print(os.getcwd())

/home/maradonam/Implementation/Anomaly/TAD-main


In [15]:
import scipy
import itertools as it
import string
import re
import random

import tensorflow as tf

import numpy as np
import pandas as pd

from scipy.stats import pearsonr
from scipy import spatial
from tqdm import tqdm 

from tensorflow.keras import Model, layers, optimizers, regularizers, callbacks, activations

from utils.evaluator import evaluate, set_thresholds, compute_traditional_metrics
from utils.evaluator_seg import compute_metrics
from utils.data_loader import _create_sequences
# Univariate
from utils.data_loader import load_kpi, load_IoT_fridge
# Multivariate
from utils.data_loader import load_samsung, load_energy, load_unsw, load_IoT_modbus

# THESE LINES ARE FOR REPRODUCIBILITY
seed = 7
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [16]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.7
tf_config.allow_soft_placement = True

In [17]:
os.chdir('/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly/')
print(os.getcwd())

/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly


In [18]:
import logging
import os
import pickle
import sys
import time
import warnings
from argparse import ArgumentParser
from pprint import pformat, pprint

import numpy as np
import tensorflow as tf
from tfsnippet.examples.utils import MLResults, print_with_title
from tfsnippet.scaffold import VariableSaver
from tfsnippet.utils import get_variables_as_dict, register_config_arguments, Config

from omni_anomaly.eval_methods import pot_eval, bf_search
from omni_anomaly.model import OmniAnomaly
from omni_anomaly.prediction import Predictor
from omni_anomaly.training import Trainer
from omni_anomaly.utils import get_data_dim, get_data, save_z
import main

In [19]:
class ExpConfig(Config):
    # dataset configuration
    dataset = "machine-1-1"
    x_dim = get_data_dim(dataset)

    # model architecture configuration
    use_connected_z_q = True
    use_connected_z_p = True

    # model parameters
    z_dim = 3
    rnn_cell = 'GRU'  # 'GRU', 'LSTM' or 'Basic'
    rnn_num_hidden = 500
    window_length = 36
    stride = 1
    dense_dim = 500
    posterior_flow_type = 'nf'  # 'nf' or None
    nf_layers = 20  # for nf
    max_epoch = 10
    train_start = 0
    max_train_size = None  # `None` means full train set
    batch_size = 50
    l2_reg = 0.0001
    initial_lr = 0.001
    lr_anneal_factor = 0.5
    lr_anneal_epoch_freq = 40
    lr_anneal_step_freq = None
    std_epsilon = 1e-4

    # evaluation parameters
    test_n_z = 1
    test_batch_size = 50
    test_start = 0
    max_test_size = None  # `None` means full test set

    # the range and step-size for score for searching best-f1
    # may vary for different dataset
    bf_search_min = -400.
    bf_search_max = 400.
    bf_search_step_size = 1.

    valid_step_freq = 50 # 100
    gradient_clip_norm = 10.

    early_stop = True  # whether to apply early stop method
    level = 0.01

    # outputs config
    save_z = False  # whether to save sampled z in hidden space
    get_score_on_dim = True  # whether to get score on dim. If `True`, the score will be a 2-dim ndarray
    save_dir = '/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly/model'
    restore_dir = None  # If not None, restore variables from this dir
    result_dir = '/home/maradonam/Implementation/Anomaly/TAD-main/baselines/OmniAnomaly/result'  # Where to save the result file
    train_score_filename = 'train_score.pkl'
    test_score_filename = 'test_score.pkl'

In [20]:
def OmniAnomaly_seg(data):
    x_train, x_valid, x_test, y_valid, y_test = data

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        anomaly_scores = main(data)
    
    return -anomaly_scores # Inverse of "The larger `reconstruction probability`, the less likely a point is anomaly" to directly indicate the severity of anomaly.

In [21]:
def train_OmniAnomaly(x_train, x_valid, x_test, y_valid, y_test, label_segment, segmentation=False):
    tf.keras.backend.clear_session()

    logging.basicConfig(
        level='INFO',
        format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
    )

    # construct the model under `variable_scope` named 'model'
    with tf.variable_scope('model') as model_vs:
        model = OmniAnomaly(config=config, name="model")

        # construct the trainer
        trainer = Trainer(model=model,
                          model_vs=model_vs,
                          max_epoch=config.max_epoch,
                          batch_size=config.batch_size,
                          valid_batch_size=config.test_batch_size,
                          initial_lr=config.initial_lr,
                          lr_anneal_epochs=config.lr_anneal_epoch_freq,
                          lr_anneal_factor=config.lr_anneal_factor,
                          grad_clip_norm=config.gradient_clip_norm,
                          valid_step_freq=config.valid_step_freq)

        # construct the predictor
        predictor = Predictor(model, batch_size=config.batch_size, n_z=config.test_n_z, last_point_only=False)

        with tf.Session(config=tf_config).as_default():

            if config.restore_dir is not None:
                # Restore variables from `save_dir`.
                saver = VariableSaver(get_variables_as_dict(model_vs), config.restore_dir)
                saver.restore()

            if config.max_epoch > 0:
                # train the model
                train_start = time.time()
                best_valid_metrics = trainer.fit((x_train, x_valid))
                train_time = (time.time() - train_start) / config.max_epoch
                best_valid_metrics.update({
                    'train_time': train_time
                })
            else:
                best_valid_metrics = {}

            train_score, train_z, train_pred_speed = predictor.get_score(x_train)

            if config.save_z:
                save_z(train_z, 'train_z')

            if x_test is not None:
                # get score of test set
                test_start = time.time()
                test_score, test_z, pred_speed = predictor.get_score(x_test)
                test_time = time.time() - test_start
                if config.save_z:
                    save_z(test_z, 'test_z')
                best_valid_metrics.update({
                    'pred_time': pred_speed,
                    'pred_total_time': test_time
                })
                
                if y_test is not None and len(y_test) >= len(test_score):
                    if config.get_score_on_dim:
                        # get the joint score
                        test_score = -np.sum(test_score, axis=-1)
                        train_score = -np.sum(train_score, axis=-1)
                        
                        if segmentation == False:
                            return compute_traditional_metrics(test_score, _create_sequences(y_test, config.window_length, config.stride))
                        else:
                            return compute_metrics(test_score, _create_sequences(y_test, config.window_length, config.stride), label_segment)

In [22]:
# get config obj
config = ExpConfig()
os.chdir('/home/maradonam/Implementation/Anomaly/TAD-main/')
result_path = './results'

---

# BASE

## Traditional Metric Evaluation

### SAMSUNG

In [23]:
samsung_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [25]:
data = load_samsung(0, 1, -0.7, 3, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        samsung_scores['dataset'].append(f'SAMSUNG{i}')
        samsung_scores['avg_f1'].append(np.average(test_scores['f1']))
        samsung_scores['avg_p'].append(np.average(test_scores['precision']))
        samsung_scores['avg_r'].append(np.average(test_scores['recall']))
        samsung_scores['max_f1'].append(np.max(test_scores['f1']))
        samsung_scores['max_p'].append(np.max(test_scores['precision']))
        samsung_scores['max_r'].append(np.max(test_scores['recall']))
        samsung_scores['pr_auc'].append(test_scores['pr_auc'])
        samsung_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/3 [00:00<?, ?it/s]

train_values: (21600, 8)
[Epoch 1/10, Step 50] step time: 0.3768s (±1.417s); train time: 11.43s; valid time: 7.456s; loss: 5.88436 (±1.82518); valid loss: 5.80165 (*)
[Epoch 1/10, Step 100] step time: 0.2068s (±0.8129s); train time: 4.549s; valid time: 5.808s; loss: 4.98839 (±0.675005); valid loss: 4.65071 (*)
[Epoch 1/10, Step 150] step time: 0.1871s (±0.666s); train time: 4.618s; valid time: 4.758s; loss: 4.30645 (±0.566436); valid loss: 8.71595
[Epoch 1/10, Step 200] step time: 0.1925s (±0.7601s); train time: 4.21s; valid time: 5.432s; loss: 5.13758 (±1.12227); valid loss: 4.47286 (*)
[Epoch 1/10, Step 250] step time: 0.1762s (±0.6517s); train time: 4.168s; valid time: 4.659s; loss: 4.81303 (±0.653744); valid loss: 4.71426
[Epoch 1/10, Step 300] step time: 0.1816s (±0.6551s); train time: 4.417s; valid time: 4.684s; loss: 4.96552 (±0.87515); valid loss: 5.40518
[Epoch 1/10, Step 350] step time: 0.1771s (±0.6398s); train time: 4.302s; valid time: 4.576s; loss: 4.5678 (±0.568272); vali

2021-12-27 12:50:41,359 [INFO] tensorflow: Restoring parameters from /tmp/tmpg1ttmow6/variables.dat-4300


------------------------------ testing ------------------------------


 33%|███▎      | 1/3 [14:44<29:28, 884.23s/it]

...

 67%|██████▋   | 2/3 [29:03<14:36, 876.73s/it]

train_values: (21600, 8)
[Epoch 1/10, Step 50] step time: 0.3622s (±1.373s); train time: 11.11s; valid time: 7.046s; loss: 6.49661 (±2.67309); valid loss: 4.96694 (*)
[Epoch 1/10, Step 100] step time: 0.1838s (±0.7128s); train time: 4.121s; valid time: 5.09s; loss: 5.39 (±2.02329); valid loss: 4.86263 (*)
[Epoch 1/10, Step 150] step time: 0.1693s (±0.6103s); train time: 4.131s; valid time: 4.355s; loss: 4.52316 (±0.597176); valid loss: 5.20305
[Epoch 1/10, Step 200] step time: 0.1843s (±0.7217s); train time: 4.08s; valid time: 5.155s; loss: 4.19362 (±0.53187); valid loss: 4.299 (*)
[Epoch 1/10, Step 250] step time: 0.184s (±0.7177s); train time: 4.1s; valid time: 5.118s; loss: 3.99121 (±0.464113); valid loss: 4.02582 (*)
[Epoch 1/10, Step 300] step time: 0.185s (±0.7159s); train time: 4.15s; valid time: 5.117s; loss: 4.18481 (±1.02097); valid loss: 3.78183 (*)
[Epoch 1/10, Step 350] step time: 0.1851s (±0.7123s); train time: 4.184s; valid time: 5.089s; loss: 4.36217 (±0.597256); valid 

2021-12-27 13:19:41,118 [INFO] tensorflow: Restoring parameters from /tmp/tmphpdq81ny/variables.dat-4300


------------------------------ testing ------------------------------


100%|██████████| 3/3 [43:51<00:00, 879.99s/it]


In [26]:
samsung_results = pd.DataFrame(samsung_scores)
samsung_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/samsung_Omni.csv', index=False)

In [27]:
samsung_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,SAMSUNG0,0.167918,0.711478,0.262536,1.000000,1.0,0.322388,0.243979,0.630257
1,SAMSUNG1,0.096748,0.799615,0.140774,1.000000,1.0,0.202740,0.180719,0.765563
2,SAMSUNG2,0.070119,0.055247,0.487601,0.108696,1.0,0.105850,0.049061,0.626015


### AIOPS KPI

In [28]:
kpi_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [29]:
data = load_kpi(0, 1, -0.7, 3, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 60
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        kpi_scores['dataset'].append(f'KPI{i}')
        kpi_scores['avg_f1'].append(np.average(test_scores['f1']))
        kpi_scores['avg_p'].append(np.average(test_scores['precision']))
        kpi_scores['avg_r'].append(np.average(test_scores['recall']))
        kpi_scores['max_f1'].append(np.max(test_scores['f1']))
        kpi_scores['max_p'].append(np.max(test_scores['precision']))
        kpi_scores['max_r'].append(np.max(test_scores['recall']))
        kpi_scores['pr_auc'].append(test_scores['pr_auc'])
        kpi_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/1 [00:00<?, ?it/s]

train_values: (66822, 1)
[Epoch 1/10, Step 50] step time: 0.8365s (±3.855s); train time: 16.27s; valid time: 25.65s; loss: 4.83933 (±1.24147); valid loss: 4.09548 (*)
[Epoch 1/10, Step 100] step time: 0.5459s (±3.027s); train time: 5.686s; valid time: 21.63s; loss: 4.05239 (±2.38664); valid loss: 4.48164
[Epoch 1/10, Step 150] step time: 0.5808s (±3.271s); train time: 5.692s; valid time: 23.36s; loss: 9.56418 (±36.4621); valid loss: 3.63381 (*)
[Epoch 1/10, Step 200] step time: 0.5483s (±3.051s); train time: 5.643s; valid time: 21.79s; loss: 8.55458 (±13.2029); valid loss: 4.79357
[Epoch 1/10, Step 250] step time: 0.5536s (±3.08s); train time: 5.698s; valid time: 22s; loss: 5.2768 (±4.49442); valid loss: 4.51606
[Epoch 1/10, Step 300] step time: 0.5585s (±3.119s); train time: 5.66s; valid time: 22.28s; loss: 56.2868 (±211.203); valid loss: 4.79089
[Epoch 1/10, Step 350] step time: 0.5714s (±3.204s); train time: 5.696s; valid time: 22.89s; loss: 4.58261 (±7.24367); valid loss: 2.9385 (*

2021-12-27 15:26:39,705 [INFO] tensorflow: Restoring parameters from /tmp/tmpbjl97ipu/variables.dat-13300


------------------------------ testing ------------------------------


100%|██████████| 1/1 [2:11:03<00:00, 7863.55s/it]


In [30]:
kpi_results = pd.DataFrame(kpi_scores)
kpi_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/KPI_Omni.csv', index=False)

In [31]:
kpi_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,KPI0,0.090012,0.992875,0.054506,1.0,1.0,0.84753,0.89844,0.96522


### IoT_modbus

In [32]:
modbus_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [33]:
data = load_IoT_modbus(0, 1, -0.7, 3, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        modbus_scores['dataset'].append(f'IoT_modbus{i}')
        modbus_scores['avg_f1'].append(np.average(test_scores['f1']))
        modbus_scores['avg_p'].append(np.average(test_scores['precision']))
        modbus_scores['avg_r'].append(np.average(test_scores['recall']))
        modbus_scores['max_f1'].append(np.max(test_scores['f1']))
        modbus_scores['max_p'].append(np.max(test_scores['precision']))
        modbus_scores['max_r'].append(np.max(test_scores['recall']))
        modbus_scores['pr_auc'].append(test_scores['pr_auc'])
        modbus_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/1 [00:00<?, ?it/s]

train_values: (15332, 4)
[Epoch 1/10, Step 50] step time: 0.5043s (±2.194s); train time: 11.01s; valid time: 14.24s; loss: 5.26462 (±0.797224); valid loss: 4.81284 (*)
[Epoch 1/10, Step 100] step time: 0.3267s (±1.714s); train time: 4.106s; valid time: 12.24s; loss: 4.81152 (±0.24068); valid loss: 4.76067 (*)
[Epoch 1/10, Step 150] step time: 0.3127s (±1.617s); train time: 4.1s; valid time: 11.55s; loss: 5.19548 (±0.762421); valid loss: 4.898
[Epoch 1/10, Step 200] step time: 0.3262s (±1.711s); train time: 4.108s; valid time: 12.22s; loss: 5.061 (±0.35677); valid loss: 4.4971 (*)
[Epoch 1/10, Step 250] step time: 0.3275s (±1.723s); train time: 4.085s; valid time: 12.31s; loss: 4.95351 (±0.699427); valid loss: 4.46489 (*)
[Epoch 1/10, Step 300] step time: 0.3264s (±1.712s); train time: 4.1s; valid time: 12.24s; loss: 4.55015 (±1.21598); valid loss: 3.53502 (*)
train_values: (15332, 4)
[Epoch 2/10, Step 350, ETA 15m 50.28s] step time: 0.3365s (±1.781s); train time: 3.727s; valid time: 12

2021-12-27 15:49:21,206 [INFO] tensorflow: Restoring parameters from /tmp/tmpr2ezlo3o/variables.dat-2750


------------------------------ testing ------------------------------


100%|██████████| 1/1 [22:22<00:00, 1342.13s/it]


In [34]:
modbus_results = pd.DataFrame(modbus_scores)
modbus_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/modbus_Omni.csv', index=False)

In [35]:
modbus_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,IoT_modbus0,0.417154,0.718587,0.487525,1.0,1.0,0.784144,0.639431,0.498499


### AIHub Energy

In [36]:
energy_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [37]:
data = load_energy(0, 1, -0.7, 3, decomposition=False, segmentation=True)

for i in tqdm(range(len(data['x_train']))):
    valid_idx = int(data['x_test'][i].shape[0] * 0.3)
    train, valid, test = data['x_train'][i], data['x_test'][i][:valid_idx], data['x_test'][i][valid_idx:]
    y_valid, y_test = data['y_test'][i][:valid_idx], data['y_test'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 60
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        energy_scores['dataset'].append(f'Energy{i}')
        energy_scores['avg_f1'].append(np.average(test_scores['f1']))
        energy_scores['avg_p'].append(np.average(test_scores['precision']))
        energy_scores['avg_r'].append(np.average(test_scores['recall']))
        energy_scores['max_f1'].append(np.max(test_scores['f1']))
        energy_scores['max_p'].append(np.max(test_scores['precision']))
        energy_scores['max_r'].append(np.max(test_scores['recall']))
        energy_scores['pr_auc'].append(test_scores['pr_auc'])
        energy_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/7 [00:00<?, ?it/s]

----------------------Combined_LabelledData_112_전압고조파평균.json TRAIN DATA----------------------
----------------------Combined_LabelledData_112_전압고조파평균.json VALIDATION DATA----------------------


 14%|█▍        | 1/7 [12:52<1:17:17, 772.91s/it]

----------------------Combined_LabelledData_127_역률평균.json TRAIN DATA----------------------
----------------------Combined_LabelledData_127_역률평균.json VALIDATION DATA----------------------


 29%|██▊       | 2/7 [23:08<1:00:28, 725.73s/it]

...

 86%|████████▌ | 6/7 [2:24:12<25:00, 1500.56s/it]

train_values: (53271, 32)
[Epoch 1/10, Step 50] step time: 0.4144s (±1.477s); train time: 14.16s; valid time: 6.663s; loss: 22.284 (±78.7496); valid loss: 8.66643 (*)
[Epoch 1/10, Step 100] step time: 0.1857s (±0.5023s); train time: 5.734s; valid time: 3.587s; loss: 1.41334 (±4.46675); valid loss: 23.8041
[Epoch 1/10, Step 150] step time: 0.1852s (±0.4997s); train time: 5.727s; valid time: 3.569s; loss: 1.86968 (±6.7983); valid loss: 9.14813
[Epoch 1/10, Step 200] step time: 0.1863s (±0.5006s); train time: 5.784s; valid time: 3.577s; loss: 1.47851 (±7.21886); valid loss: 23.9165
[Epoch 1/10, Step 250] step time: 0.1868s (±0.5006s); train time: 5.807s; valid time: 3.573s; loss: -0.816244 (±2.5549); valid loss: 25.8936
[Epoch 1/10, Step 300] step time: 0.1849s (±0.4985s); train time: 5.726s; valid time: 3.557s; loss: -0.0573071 (±3.39903); valid loss: 13.7938
[Epoch 1/10, Step 350] step time: 0.1864s (±0.5002s); train time: 5.787s; valid time: 3.573s; loss: -1.86176 (±6.22448); valid los

2021-12-27 20:37:07,909 [INFO] tensorflow: Restoring parameters from /tmp/tmpov0yteet/variables.dat-10050


------------------------------ testing ------------------------------


100%|██████████| 7/7 [2:59:58<00:00, 1694.33s/it]


In [38]:
energy_results = pd.DataFrame(energy_scores)
energy_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/Energy_Omni.csv', index=False)

In [39]:
energy_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,Energy0,0.027285,0.022615,0.037423,0.515346,1.0,0.680169,0.318492,0.139747
1,Energy1,0.035736,0.137900,0.036422,0.912281,1.0,0.775046,0.740814,0.651931
2,Energy2,0.058156,0.998170,0.037230,1.000000,1.0,0.993209,0.973028,0.152299
3,Energy3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.000000
4,Energy4,0.166971,0.988854,0.125667,1.000000,1.0,0.923969,0.964409,0.874421
5,Energy5,0.198324,0.997690,0.155928,1.000000,1.0,0.998109,0.995076,0.284036
6,Energy6,0.212733,0.969977,0.195037,1.000000,1.0,0.947872,0.972489,0.961608


---

---

# Deep Residual-based

### SAMSUNG

In [40]:
samsung_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [41]:
save_file_dir = './datasets/samsung/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

for i in tqdm(range(len(data['x_train_resid_long']))):
    valid_idx = int(data['x_test_resid_long'][i].shape[0] * 0.3)
    train, valid, test = data['x_train_resid_long'][i], data['x_test_resid_long'][i][:valid_idx], data['x_test_resid_long'][i][valid_idx:]
    y_valid, y_test = data['label_seq'][i][:valid_idx], data['label_seq'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        samsung_scores['dataset'].append(f'SAMSUNG_{i}')
        samsung_scores['avg_f1'].append(np.average(test_scores['f1']))
        samsung_scores['avg_p'].append(np.average(test_scores['precision']))
        samsung_scores['avg_r'].append(np.average(test_scores['recall']))
        samsung_scores['max_f1'].append(np.max(test_scores['f1']))
        samsung_scores['max_p'].append(np.max(test_scores['precision']))
        samsung_scores['max_r'].append(np.max(test_scores['recall']))
        samsung_scores['pr_auc'].append(test_scores['pr_auc'])
        samsung_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/3 [00:00<?, ?it/s]

train_values: (21600, 8)
[Epoch 1/10, Step 50] step time: 0.3551s (±1.338s); train time: 10.91s; valid time: 6.884s; loss: 14.7781 (±36.3119); valid loss: -1.37438 (*)
[Epoch 1/10, Step 100] step time: 0.1833s (±0.7129s); train time: 4.094s; valid time: 5.088s; loss: -7.88938 (±3.84274); valid loss: -9.95328 (*)
[Epoch 1/10, Step 150] step time: 0.1687s (±0.6088s); train time: 4.102s; valid time: 4.348s; loss: -12.8109 (±2.88381); valid loss: -6.2609
[Epoch 1/10, Step 200] step time: 0.1817s (±0.7017s); train time: 4.087s; valid time: 5.013s; loss: -13.6894 (±3.37489); valid loss: -13.4354 (*)
[Epoch 1/10, Step 250] step time: 0.1821s (±0.704s); train time: 4.09s; valid time: 5.028s; loss: -15.7957 (±1.98685); valid loss: -14.7701 (*)
[Epoch 1/10, Step 300] step time: 0.1817s (±0.6991s); train time: 4.104s; valid time: 4.995s; loss: -15.9974 (±2.50045); valid loss: -14.9475 (*)
[Epoch 1/10, Step 350] step time: 0.1691s (±0.606s); train time: 4.149s; valid time: 4.328s; loss: -15.742 (±

2021-12-28 10:16:09,158 [INFO] tensorflow: Restoring parameters from /tmp/tmpn47gs995/variables.dat-3950


------------------------------ testing ------------------------------


 33%|███▎      | 1/3 [14:13<28:27, 853.76s/it]

...

 67%|██████▋   | 2/3 [28:13<14:09, 849.60s/it]

train_values: (21600, 8)
[Epoch 1/10, Step 50] step time: 0.3448s (±1.29s); train time: 10.38s; valid time: 6.904s; loss: 30.9911 (±65.7008); valid loss: 1.87222 (*)
[Epoch 1/10, Step 100] step time: 0.184s (±0.7125s); train time: 4.132s; valid time: 5.083s; loss: -3.9861 (±11.5977); valid loss: -6.31599 (*)
[Epoch 1/10, Step 150] step time: 0.1832s (±0.7123s); train time: 4.084s; valid time: 5.089s; loss: -12.5955 (±2.38488); valid loss: -14.6483 (*)
[Epoch 1/10, Step 200] step time: 0.1701s (±0.6139s); train time: 4.143s; valid time: 4.376s; loss: -14.3602 (±1.55563); valid loss: -13.491
[Epoch 1/10, Step 250] step time: 0.1694s (±0.6109s); train time: 4.119s; valid time: 4.367s; loss: -15.6519 (±1.93223); valid loss: -13.2271
[Epoch 1/10, Step 300] step time: 0.185s (±0.7238s); train time: 4.097s; valid time: 5.171s; loss: -16.8778 (±1.67568); valid loss: -14.9432 (*)
[Epoch 1/10, Step 350] step time: 0.1841s (±0.7142s); train time: 4.117s; valid time: 5.106s; loss: -16.5927 (±1.719

2021-12-28 10:44:22,002 [INFO] tensorflow: Restoring parameters from /tmp/tmpx2a09fm9/variables.dat-4150


------------------------------ testing ------------------------------


100%|██████████| 3/3 [42:27<00:00, 850.89s/it]


In [42]:
samsung_results = pd.DataFrame(samsung_scores)
samsung_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/samsung_Omni_CAD.csv', index=False)

In [43]:
samsung_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,SAMSUNG_0,0.134268,0.978050,0.090832,1.0,1.0,0.543840,0.457127,0.692165
1,SAMSUNG_1,0.130255,0.990842,0.081551,1.0,1.0,0.494382,0.373060,0.662121
2,SAMSUNG_2,0.213741,0.926998,0.164297,1.0,1.0,0.413002,0.277611,0.418148


### AIOPS KPI

In [44]:
kpi_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [45]:
save_file_dir = './datasets/KPI/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

for i in tqdm(range(len(data['x_train_resid_long']))):
    valid_idx = int(data['x_test_resid_long'][i].shape[0] * 0.3)
    train, valid, test = data['x_train_resid_long'][i], data['x_test_resid_long'][i][:valid_idx], data['x_test_resid_long'][i][valid_idx:]
    y_valid, y_test = data['label_seq'][i][:valid_idx], data['label_seq'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)

        kpi_scores['dataset'].append('KPI')
        kpi_scores['avg_f1'].append(np.average(test_scores['f1']))
        kpi_scores['avg_p'].append(np.average(test_scores['precision']))
        kpi_scores['avg_r'].append(np.average(test_scores['recall']))
        kpi_scores['max_f1'].append(np.max(test_scores['f1']))
        kpi_scores['max_p'].append(np.max(test_scores['precision']))
        kpi_scores['max_r'].append(np.max(test_scores['recall']))
        kpi_scores['pr_auc'].append(test_scores['pr_auc'])
        kpi_scores['roc_auc'].append(test_scores['roc_auc'])       

  0%|          | 0/1 [00:00<?, ?it/s]

train_values: (66822, 1)
[Epoch 1/10, Step 50] step time: 0.545s (±2.507s); train time: 10.44s; valid time: 16.9s; loss: 2.46498e+06 (±1.72384e+07); valid loss: 4.17143 (*)
[Epoch 1/10, Step 100] step time: 0.37s (±2.021s); train time: 4.084s; valid time: 14.43s; loss: 5.72442 (±5.09014); valid loss: 4.58697
[Epoch 1/10, Step 150] step time: 0.37s (±2.02s); train time: 4.084s; valid time: 14.43s; loss: 6.6574 (±10.9999); valid loss: 4.46804
[Epoch 1/10, Step 200] step time: 0.3705s (±2.022s); train time: 4.094s; valid time: 14.45s; loss: 73.118 (±368.629); valid loss: 5.21035
[Epoch 1/10, Step 250] step time: 0.3693s (±2.01s); train time: 4.125s; valid time: 14.36s; loss: 5.01739 (±3.88441); valid loss: 4.63482
[Epoch 1/10, Step 300] step time: 0.3695s (±2.023s); train time: 4.032s; valid time: 14.46s; loss: 6.28702 (±6.66516); valid loss: 4.49277
[Epoch 1/10, Step 350] step time: 0.3869s (±2.132s); train time: 4.129s; valid time: 15.23s; loss: 4.85366 (±1.91455); valid loss: 4.09636 (

2021-12-28 12:10:08,945 [INFO] tensorflow: Restoring parameters from /tmp/tmpua1z_noa/variables.dat-13350


------------------------------ testing ------------------------------


100%|██████████| 1/1 [1:28:58<00:00, 5338.71s/it]


In [46]:
kpi_results = pd.DataFrame(kpi_scores)
kpi_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/KPI_Omni_CAD.csv', index=False)

In [47]:
kpi_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,KPI,0.066092,0.973349,0.042874,1.0,1.0,0.322898,0.286039,0.874957


### IoT_modbus

In [48]:
modbus_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [49]:
save_file_dir = './datasets/IoT_modbus/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

In [50]:
len(data['x_train_resid_long'])

1

In [51]:
data['x_train_resid_long'][0].shape

(15332, 4)

In [52]:
save_file_dir = './datasets/IoT_modbus/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

for i in tqdm(range(len(data['x_train_resid_long']))):
    valid_idx = int(data['x_test_resid_long'][i].shape[0] * 0.3)
    train, valid, test = data['x_train_resid_long'][i], data['x_test_resid_long'][i][:valid_idx], data['x_test_resid_long'][i][valid_idx:]
    y_valid, y_test = data['label_seq'][i][:valid_idx], data['label_seq'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)
        
        modbus_scores['dataset'].append('IoT_modbus')
        modbus_scores['avg_f1'].append(np.average(test_scores['f1']))
        modbus_scores['avg_p'].append(np.average(test_scores['precision']))
        modbus_scores['avg_r'].append(np.average(test_scores['recall']))
        modbus_scores['max_f1'].append(np.max(test_scores['f1']))
        modbus_scores['max_p'].append(np.max(test_scores['precision']))
        modbus_scores['max_r'].append(np.max(test_scores['recall']))
        modbus_scores['pr_auc'].append(test_scores['pr_auc'])
        modbus_scores['roc_auc'].append(test_scores['roc_auc'])

  0%|          | 0/1 [00:00<?, ?it/s]

train_values: (15332, 4)
[Epoch 1/10, Step 50] step time: 0.488s (±2.137s); train time: 10.4s; valid time: 14.03s; loss: 14441.9 (±100185); valid loss: 4.21669 (*)
[Epoch 1/10, Step 100] step time: 0.3383s (±1.789s); train time: 4.152s; valid time: 12.78s; loss: 0.747887 (±2.88393); valid loss: -1.81044 (*)
[Epoch 1/10, Step 150] step time: 0.3275s (±1.722s); train time: 4.096s; valid time: 12.3s; loss: -2.45183 (±1.40836); valid loss: -4.19516 (*)
[Epoch 1/10, Step 200] step time: 0.3144s (±1.629s); train time: 4.104s; valid time: 11.63s; loss: -3.42197 (±1.90019); valid loss: 5.46697
[Epoch 1/10, Step 250] step time: 0.3189s (±1.662s); train time: 4.094s; valid time: 11.87s; loss: -4.07636 (±1.55958); valid loss: -2.2949
[Epoch 1/10, Step 300] step time: 0.3291s (±1.733s); train time: 4.092s; valid time: 12.38s; loss: -4.52833 (±1.15231); valid loss: -5.37148 (*)
train_values: (15332, 4)
[Epoch 2/10, Step 350, ETA 15m 38.54s] step time: 0.3137s (±1.622s); train time: 3.733s; valid ti

2021-12-28 12:31:17,605 [INFO] tensorflow: Restoring parameters from /tmp/tmpackz1fbt/variables.dat-3050


------------------------------ testing ------------------------------


100%|██████████| 1/1 [21:04<00:00, 1264.98s/it]


In [53]:
modbus_results = pd.DataFrame(modbus_scores)
modbus_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/modbus_Omni_CAD.csv', index=False)

In [54]:
modbus_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,IoT_modbus,0.145589,0.531641,0.118527,1.0,1.0,0.783499,0.548915,0.311679


### AIHub Energy

In [55]:
energy_scores = {'dataset': [], 'avg_f1': [], 'avg_p': [], 'avg_r': [], 'max_p': [], 'max_r': [],  'max_f1': [], 'pr_auc': [], 'roc_auc': []}

In [56]:
save_file_dir = './datasets/aihub/deep_residual'
with open(f'{save_file_dir}/deep_resid.pkl', 'rb') as f:
    data = pickle.load(f)

for i in tqdm(range(len(data['x_train_resid_long']))):
    valid_idx = int(data['x_test_resid_long'][i].shape[0] * 0.3)
    train, valid, test = data['x_train_resid_long'][i], data['x_test_resid_long'][i][:valid_idx], data['x_test_resid_long'][i][valid_idx:]
    y_valid, y_test = data['label_seq'][i][:valid_idx], data['label_seq'][i][valid_idx:]
    label_segment = data['y_segment_test'][i][valid_idx:]

    with warnings.catch_warnings():
        # suppress DeprecationWarning from NumPy caused by codes in TensorFlow-Probability
        warnings.filterwarnings("ignore", category=DeprecationWarning, module='numpy')
        config.x_dim = train.shape[-1]
        config.window_length = 36
        test_scores = train_OmniAnomaly(train, valid, test, y_valid, y_test, label_segment, segmentation=False)
        
        energy_scores['dataset'].append('Energy')
        energy_scores['avg_f1'].append(np.average(test_scores['f1']))
        energy_scores['avg_p'].append(np.average(test_scores['precision']))
        energy_scores['avg_r'].append(np.average(test_scores['recall']))
        energy_scores['max_f1'].append(np.max(test_scores['f1']))
        energy_scores['max_p'].append(np.max(test_scores['precision']))
        energy_scores['max_r'].append(np.max(test_scores['recall']))
        energy_scores['pr_auc'].append(test_scores['pr_auc'])
        energy_scores['roc_auc'].append(test_scores['roc_auc'])    

  0%|          | 0/7 [00:00<?, ?it/s]

train_values: (38880, 32)
[Epoch 1/10, Step 50] step time: 0.2959s (±1.08s); train time: 10.94s; valid time: 3.923s; loss: 36.0874 (±345.112); valid loss: -54.116 (*)
[Epoch 1/10, Step 100] step time: 0.1128s (±0.2062s); train time: 4.188s; valid time: 1.472s; loss: -50.088 (±11.6376); valid loss: 2.48022
[Epoch 1/10, Step 150] step time: 0.1251s (±0.3003s); train time: 4.129s; valid time: 2.143s; loss: -55.528 (±23.8188); valid loss: -82.6161 (*)
[Epoch 1/10, Step 200] step time: 0.1109s (±0.2002s); train time: 4.133s; valid time: 1.428s; loss: -68.1004 (±5.78002); valid loss: -80.3106
[Epoch 1/10, Step 250] step time: 0.1261s (±0.3029s); train time: 4.16s; valid time: 2.161s; loss: -69.6546 (±5.22372); valid loss: -89.6465 (*)
[Epoch 1/10, Step 300] step time: 0.1113s (±0.2011s); train time: 4.153s; valid time: 1.43s; loss: -71.4923 (±6.33126); valid loss: -80.7203
[Epoch 1/10, Step 350] step time: 0.111s (±0.2021s); train time: 4.12s; valid time: 1.446s; loss: -75.6143 (±6.17104); v

2021-12-28 12:50:51,189 [INFO] tensorflow: Restoring parameters from /tmp/tmp5hog9g75/variables.dat-2650


------------------------------ testing ------------------------------


 14%|█▍        | 1/7 [15:58<1:35:51, 958.57s/it]

...

 86%|████████▌ | 6/7 [1:42:58<17:49, 1069.44s/it]

train_values: (53271, 32)
[Epoch 1/10, Step 50] step time: 0.306s (±1.095s); train time: 10.52s; valid time: 4.861s; loss: 41580 (±273045); valid loss: -36.9442 (*)
[Epoch 1/10, Step 100] step time: 0.13s (±0.3338s); train time: 4.129s; valid time: 2.389s; loss: -43.1635 (±12.2104); valid loss: -35.8935
[Epoch 1/10, Step 150] step time: 0.1304s (±0.3379s); train time: 4.126s; valid time: 2.415s; loss: -58.0056 (±7.44617); valid loss: -33.7914
[Epoch 1/10, Step 200] step time: 0.1438s (±0.4279s); train time: 4.152s; valid time: 3.056s; loss: -59.6097 (±4.51092); valid loss: -44.7706 (*)
[Epoch 1/10, Step 250] step time: 0.1307s (±0.3391s); train time: 4.136s; valid time: 2.419s; loss: -64.5188 (±7.10244); valid loss: -20.9125
[Epoch 1/10, Step 300] step time: 0.1466s (±0.4427s); train time: 4.195s; valid time: 3.157s; loss: -63.9586 (±7.12382); valid loss: -48.1924 (*)
[Epoch 1/10, Step 350] step time: 0.1301s (±0.3306s); train time: 4.16s; valid time: 2.365s; loss: -66.2504 (±5.35967);

2021-12-28 14:42:27,090 [INFO] tensorflow: Restoring parameters from /tmp/tmpjxi10i3t/variables.dat-9900


------------------------------ testing ------------------------------


100%|██████████| 7/7 [2:08:07<00:00, 1201.47s/it]


In [57]:
energy_results = pd.DataFrame(energy_scores)
energy_results.groupby('dataset', as_index=False).mean().to_csv(f'{result_path}/Energy_Omni_CAD.csv', index=False)

In [58]:
energy_results

,dataset,avg_f1,avg_p,avg_r,max_p,max_r,max_f1,pr_auc,roc_auc
0,Energy,0.000640,0.000471,0.000999,0.471061,1.0,0.640437,0.235530,0.383618
1,Energy,0.009375,0.047940,0.007245,0.620690,1.0,0.740147,0.543517,0.486929
2,Energy,0.742122,0.977284,0.702979,1.000000,1.0,0.983231,0.987876,0.718055
3,Energy,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.000000
4,Energy,0.203096,0.972126,0.127030,1.000000,1.0,0.888330,0.938587,0.838435
5,Energy,0.098564,0.993071,0.067104,1.000000,1.0,0.992565,0.978257,0.288828
6,Energy,0.145924,0.996586,0.090572,1.000000,1.0,0.743409,0.797842,0.654750


---